In [1]:
import re
import numpy as np

def read_pgm(filename, byteorder='>'):
    """Return image data from a raw PGM file as np array.

    Format specification: http://netpbm.sourceforge.net/doc/pgm.html

    """
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))

def resizeImage(image, newSize):
    a = image.shape[0]
    b = newSize
    ratio = a/b
    resizeAToB= np.zeros((a,b))
    for i in range(a):
        for j in range(b): 
            if i//ratio == j:
                resizeAToB[i,j] = 1. / ratio
    return np.matmul(resizeAToB.T, np.matmul(image, resizeAToB))

In [ ]:
from subprocess import Popen, PIPE

def flatten_list_of_strings(args):
    return reduce(lambda x,y: x +' '+y, args)

def run_exp(input_image_path, discretization_n, amount_of_rays, random_noise, exp_list):
    tp_exec = '../build/tp3'
    exp_name = flatten_list_of_strings(exp_list)
    
    output_image_name = "{}_{}_celdas_{}_ruido_{}_rayos".format(
        exp_name, discretization_n, random_noise, amount_of_rays)
    output_image_dir = "../results/"
    output_image_path = output_image_dir + "\"" + output_image_name + '.pgm\"'
    output_image_path = output_image_path.replace(' ', '_')
    
    args = [tp_exec, "-i", str(input_image_path), "-o", str(output_image_path), 
            "-n", str(discretization_n), "-r", str(random_noise)]
    for exp in exp_list:
        if 'g15' in exp or 'g14' in exp:
            args.extend(exp.split())
            args.append(str(amount_of_rays//len(exp_list)))
        else:
            args.extend([exp, str(amount_of_rays//len(exp_list))])
        
    print(args)
    print(flatten_list_of_strings(args))
    
    run_logs = {}
    p = Popen(args, stdout=PIPE, bufsize=1)
    with p.stdout:
        for line in iter(p.stdout.readline, b''):
            try:
                key, val = line[:-1].split(': ')
                if key in ['error','debug']:
                    print(line)
                else:
                    run_logs[key] = val    
            except:
                print('COMANDO FEO: {}'.format(line))
    p.wait() # wait for the subprocess to exit
    print(output_image_path)
    output_image = read_pgm(output_image_path)  # .reshape(-1)
    process_data = {'discretization_n':float(discretization_n), 
            'random_noise': float(random_noise),
            'rayos': float(amount_of_rays),
            'image': output_image,
            'exp_name': exp_name,
            'output_image_name': output_image_name}
    
    for key in ['tiempo_cml_microseconds', 'tiempo_geometria_microseconds', 'tiempo_matrices_microseconds']:
        run_logs[key] = float(run_logs[key])
    
    return dict(run_logs, **process_data)

#### Run exp

In [ ]:
input_image_path = "../imgs_TC/tomo.pgm"
divisors_of_100 = [50]
import pandas as pd
output = []
for div in divisors_of_100:
    exp_names = [['-g4']]
    exp_names.extend([['-g14b '+focos+' -g14a'] for focos in ['50']])
    for exp in exp_names:
        for random in [0]:        
            for amount_of_rays in [500,1000, 10000]:
                output.append(run_exp(input_image_path, div, amount_of_rays, random, exp))

outputs_df = pd.DataFrame(output)
outputs_df.head()

['../build/tp3', '-i', '../imgs_TC/tomo.pgm', '-o', '../results/"-g4_50_celdas_0_ruido_500_rayos.pgm"', '-n', '50', '-r', '0', '-g4', '500']
../build/tp3 -i ../imgs_TC/tomo.pgm -o ../results/"-g4_50_celdas_0_ruido_500_rayos.pgm" -n 50 -r 0 -g4 500
debug: _rayos.size() 500

COMANDO FEO: Calculando autovector nº 1

COMANDO FEO: Calculando autovector nº 2

COMANDO FEO: Calculando autovector nº 3

COMANDO FEO: Calculando autovector nº 4

COMANDO FEO: Calculando autovector nº 5

COMANDO FEO: Calculando autovector nº 6

COMANDO FEO: Calculando autovector nº 7

COMANDO FEO: Calculando autovector nº 8

COMANDO FEO: Calculando autovector nº 9

COMANDO FEO: Calculando autovector nº 10

COMANDO FEO: Calculando autovector nº 11

COMANDO FEO: Calculando autovector nº 12

COMANDO FEO: Calculando autovector nº 13

COMANDO FEO: Calculando autovector nº 14

COMANDO FEO: Calculando autovector nº 15

COMANDO FEO: Calculando autovector nº 16

COMANDO FEO: Calculando autovector nº 17

COMANDO FEO: Calculan

COMANDO FEO: Calculando autovector nº 188

COMANDO FEO: Calculando autovector nº 189

COMANDO FEO: Calculando autovector nº 190

COMANDO FEO: Calculando autovector nº 191

COMANDO FEO: Calculando autovector nº 192

COMANDO FEO: Calculando autovector nº 193

COMANDO FEO: Calculando autovector nº 194

COMANDO FEO: Calculando autovector nº 195

COMANDO FEO: Calculando autovector nº 196

COMANDO FEO: Calculando autovector nº 197

COMANDO FEO: Calculando autovector nº 198

COMANDO FEO: Calculando autovector nº 199

COMANDO FEO: Calculando autovector nº 200

COMANDO FEO: Calculando autovector nº 201

COMANDO FEO: Calculando autovector nº 202

COMANDO FEO: Calculando autovector nº 203

COMANDO FEO: Calculando autovector nº 204

COMANDO FEO: Calculando autovector nº 205

COMANDO FEO: Calculando autovector nº 206

COMANDO FEO: Calculando autovector nº 207

COMANDO FEO: Calculando autovector nº 208

COMANDO FEO: Calculando autovector nº 209

COMANDO FEO: Calculando autovector nº 210

COMANDO FEO

COMANDO FEO: Calculando autovector nº 379

COMANDO FEO: Calculando autovector nº 380

COMANDO FEO: Calculando autovector nº 381

COMANDO FEO: Calculando autovector nº 382

COMANDO FEO: Calculando autovector nº 383

COMANDO FEO: Calculando autovector nº 384

COMANDO FEO: Calculando autovector nº 385

COMANDO FEO: Calculando autovector nº 386

COMANDO FEO: Calculando autovector nº 387

COMANDO FEO: Calculando autovector nº 388

COMANDO FEO: Calculando autovector nº 389

COMANDO FEO: Calculando autovector nº 390

COMANDO FEO: Calculando autovector nº 391

COMANDO FEO: Calculando autovector nº 392

COMANDO FEO: Calculando autovector nº 393

COMANDO FEO: Calculando autovector nº 394

COMANDO FEO: Calculando autovector nº 395

COMANDO FEO: Calculando autovector nº 396

COMANDO FEO: Calculando autovector nº 397

COMANDO FEO: Calculando autovector nº 398

COMANDO FEO: Calculando autovector nº 399

COMANDO FEO: Calculando autovector nº 400

COMANDO FEO: Calculando autovector nº 401

COMANDO FEO

COMANDO FEO: Calculando autovector nº 570

COMANDO FEO: Calculando autovector nº 571

COMANDO FEO: Calculando autovector nº 572

COMANDO FEO: Calculando autovector nº 573

COMANDO FEO: Calculando autovector nº 574

COMANDO FEO: Calculando autovector nº 575

COMANDO FEO: Calculando autovector nº 576

COMANDO FEO: Calculando autovector nº 577

COMANDO FEO: Calculando autovector nº 578

COMANDO FEO: Calculando autovector nº 579

COMANDO FEO: Calculando autovector nº 580

COMANDO FEO: Calculando autovector nº 581

COMANDO FEO: Calculando autovector nº 582

COMANDO FEO: Calculando autovector nº 583

COMANDO FEO: Calculando autovector nº 584

COMANDO FEO: Calculando autovector nº 585

COMANDO FEO: Calculando autovector nº 586

COMANDO FEO: Calculando autovector nº 587

COMANDO FEO: Calculando autovector nº 588

COMANDO FEO: Calculando autovector nº 589

COMANDO FEO: Calculando autovector nº 590

COMANDO FEO: Calculando autovector nº 591

COMANDO FEO: Calculando autovector nº 592

COMANDO FEO

COMANDO FEO: Calculando autovector nº 761

COMANDO FEO: Calculando autovector nº 762

COMANDO FEO: Calculando autovector nº 763

COMANDO FEO: Calculando autovector nº 764

COMANDO FEO: Calculando autovector nº 765

COMANDO FEO: Calculando autovector nº 766

COMANDO FEO: Calculando autovector nº 767

COMANDO FEO: Calculando autovector nº 768

COMANDO FEO: Calculando autovector nº 769

COMANDO FEO: Calculando autovector nº 770

COMANDO FEO: Calculando autovector nº 771

COMANDO FEO: Calculando autovector nº 772

COMANDO FEO: Calculando autovector nº 773

COMANDO FEO: Calculando autovector nº 774

COMANDO FEO: Calculando autovector nº 775

COMANDO FEO: Calculando autovector nº 776

COMANDO FEO: Calculando autovector nº 777

COMANDO FEO: Calculando autovector nº 778

COMANDO FEO: Calculando autovector nº 779

COMANDO FEO: Calculando autovector nº 780

COMANDO FEO: Calculando autovector nº 781

COMANDO FEO: Calculando autovector nº 782

COMANDO FEO: Calculando autovector nº 783

COMANDO FEO

COMANDO FEO: Calculando autovector nº 952

COMANDO FEO: Calculando autovector nº 953

COMANDO FEO: Calculando autovector nº 954

COMANDO FEO: Calculando autovector nº 955

COMANDO FEO: Calculando autovector nº 956

COMANDO FEO: Calculando autovector nº 957

COMANDO FEO: Calculando autovector nº 958

COMANDO FEO: Calculando autovector nº 959

COMANDO FEO: Calculando autovector nº 960

COMANDO FEO: Calculando autovector nº 961

COMANDO FEO: Calculando autovector nº 962

COMANDO FEO: Calculando autovector nº 963

COMANDO FEO: Calculando autovector nº 964

COMANDO FEO: Calculando autovector nº 965

COMANDO FEO: Calculando autovector nº 966

COMANDO FEO: Calculando autovector nº 967

COMANDO FEO: Calculando autovector nº 968

COMANDO FEO: Calculando autovector nº 969

COMANDO FEO: Calculando autovector nº 970

COMANDO FEO: Calculando autovector nº 971

COMANDO FEO: Calculando autovector nº 972

COMANDO FEO: Calculando autovector nº 973

COMANDO FEO: Calculando autovector nº 974

COMANDO FEO

COMANDO FEO: Calculando autovector nº 1140

COMANDO FEO: Calculando autovector nº 1141

COMANDO FEO: Calculando autovector nº 1142

COMANDO FEO: Calculando autovector nº 1143

COMANDO FEO: Calculando autovector nº 1144

COMANDO FEO: Calculando autovector nº 1145

COMANDO FEO: Calculando autovector nº 1146

COMANDO FEO: Calculando autovector nº 1147

COMANDO FEO: Calculando autovector nº 1148

COMANDO FEO: Calculando autovector nº 1149

COMANDO FEO: Calculando autovector nº 1150

COMANDO FEO: Calculando autovector nº 1151

COMANDO FEO: Calculando autovector nº 1152

COMANDO FEO: Calculando autovector nº 1153

COMANDO FEO: Calculando autovector nº 1154

COMANDO FEO: Calculando autovector nº 1155

COMANDO FEO: Calculando autovector nº 1156

COMANDO FEO: Calculando autovector nº 1157

COMANDO FEO: Calculando autovector nº 1158

COMANDO FEO: Calculando autovector nº 1159

COMANDO FEO: Calculando autovector nº 1160

COMANDO FEO: Calculando autovector nº 1161

COMANDO FEO: Calculando autovect

COMANDO FEO: Calculando autovector nº 1327

COMANDO FEO: Calculando autovector nº 1328

COMANDO FEO: Calculando autovector nº 1329

COMANDO FEO: Calculando autovector nº 1330

COMANDO FEO: Calculando autovector nº 1331

COMANDO FEO: Calculando autovector nº 1332

COMANDO FEO: Calculando autovector nº 1333

COMANDO FEO: Calculando autovector nº 1334

COMANDO FEO: Calculando autovector nº 1335

COMANDO FEO: Calculando autovector nº 1336

COMANDO FEO: Calculando autovector nº 1337

COMANDO FEO: Calculando autovector nº 1338

COMANDO FEO: Calculando autovector nº 1339

COMANDO FEO: Calculando autovector nº 1340

COMANDO FEO: Calculando autovector nº 1341

COMANDO FEO: Calculando autovector nº 1342

COMANDO FEO: Calculando autovector nº 1343

COMANDO FEO: Calculando autovector nº 1344

COMANDO FEO: Calculando autovector nº 1345

COMANDO FEO: Calculando autovector nº 1346

COMANDO FEO: Calculando autovector nº 1347

COMANDO FEO: Calculando autovector nº 1348

COMANDO FEO: Calculando autovect

COMANDO FEO: Calculando autovector nº 1514

COMANDO FEO: Calculando autovector nº 1515

COMANDO FEO: Calculando autovector nº 1516

COMANDO FEO: Calculando autovector nº 1517

COMANDO FEO: Calculando autovector nº 1518

COMANDO FEO: Calculando autovector nº 1519

COMANDO FEO: Calculando autovector nº 1520

COMANDO FEO: Calculando autovector nº 1521

COMANDO FEO: Calculando autovector nº 1522

COMANDO FEO: Calculando autovector nº 1523

COMANDO FEO: Calculando autovector nº 1524

COMANDO FEO: Calculando autovector nº 1525

COMANDO FEO: Calculando autovector nº 1526

COMANDO FEO: Calculando autovector nº 1527

COMANDO FEO: Calculando autovector nº 1528

COMANDO FEO: Calculando autovector nº 1529

COMANDO FEO: Calculando autovector nº 1530

COMANDO FEO: Calculando autovector nº 1531

COMANDO FEO: Calculando autovector nº 1532

COMANDO FEO: Calculando autovector nº 1533

COMANDO FEO: Calculando autovector nº 1534

COMANDO FEO: Calculando autovector nº 1535

COMANDO FEO: Calculando autovect

#### Input images

In [ ]:
from matplotlib import pyplot 
input_image_path = "../imgs_TC/tomo.pgm"
input_image = read_pgm(input_image_path, byteorder='<')
divisors = [20,25,50]
resized_input_images = {div: resizeImage(input_image, div) for div in divisors}
for div in divisors:    
    resized_input_image = resized_input_images[div]
    pyplot.imshow(resized_input_image, pyplot.cm.gray)
    pyplot.savefig('./resized_tomo{}x{}.png'.format(div,div))
    pyplot.show()
    

#### Reconstructed images

In [ ]:
image_and_metadata_headers = ['image', 'output_image_name']
for i,row in outputs_df[image_and_metadata_headers].iterrows():
    output_image_name = row['output_image_name']
    pyplot.title(output_image_name)
    pyplot.imshow(row['image'], pyplot.cm.gray)
    pyplot.show()



#### Compute mean squared error

In [ ]:
from sklearn.metrics import mean_squared_error
def mse_vs_benchmark(resized_input_images, output_img):
    output_img = output_img.reshape(-1)
    div = np.sqrt(len(output_img))
    resized_input_image = resized_input_images[div].reshape(-1)
    return mean_squared_error(resized_input_image, output_img)

from functools import partial
mse_vs_resized_input_images = partial(mse_vs_benchmark,resized_input_images)

outputs_df['mse_vs_benchmark'] = outputs_df['image'].map(mse_vs_resized_input_images)
outputs_df.head()

In [ ]:
from textwrap import wrap
def plot_param_vs_mse_per_exp(outputs_df, param_key):
    print('{} vs {} plots'.format(param_key, 'mse_vs_benchmark'))
    for exp_name in set(outputs_df['exp_name']):
        exp_filtered = outputs_df[(outputs_df['exp_name'] == exp_name)]

        exp_filtered = exp_filtered.groupby([param_key],as_index=False)['mse_vs_benchmark'].mean()

        title = 'exp_name_{}_{}_vs_{}'.format(exp_name,param_key, 'mse_vs_benchmark')
        title = "\n".join(wrap(title, 30))
        exp_filtered.plot(param_key, 'mse_vs_benchmark',title=title)

def plot_param_vs_times_per_exp(outputs_df, param_key):
    time_keys = ['tiempo_cml_microseconds', 'tiempo_geometria_microseconds', 'tiempo_matrices_microseconds']
    print('{} vs time plots'.format(param_key))
    for exp_name in set(outputs_df['exp_name']):
        exp_filtered = outputs_df[(outputs_df['exp_name'] == exp_name)]
        exp_filtered = exp_filtered.groupby([param_key],as_index=False)[time_keys].mean()

        title = 'exp_name_{}_{}_vs_time'.format(exp_name,param_key)
        title = "\n".join(wrap(title, 30))
        exp_filtered.plot(param_key, time_keys,title=title)

plot_param_vs_times_per_exp(outputs_df, 'rayos')

In [ ]:
plot_param_vs_mse_per_exp(outputs_df, 'rayos')
plot_param_vs_mse_per_exp(outputs_df, 'discretization_n')
plot_param_vs_mse_per_exp(outputs_df, 'random_noise')

In [ ]:
pd.options.mode.chained_assignment = None
from textwrap import wrap
time_keys = ['tiempo_cml_microseconds', 'tiempo_geometria_microseconds', 'tiempo_matrices_microseconds']

for exp_name in set(outputs_df['exp_name']):
    exp_filtered = outputs_df[(outputs_df['exp_name'] == exp_name)]
    exp_filtered['time'] = exp_filtered['tiempo_cml_microseconds'] + exp_filtered['tiempo_geometria_microseconds'] + exp_filtered['tiempo_matrices_microseconds']
    title =  '{}_time_and_noise_vs_mse'.format(exp_name)
    title = "\n".join(wrap(title, 30))
    fig, ax = plt.subplots()
    exp_filtered.plot.scatter(x='rayos',y='random_noise', c='mse_vs_benchmark',title=title, legend=True, cmap="cool",ax=ax);
